In [12]:
import pandas as pd
pd.set_option('display.max_rows', 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import copy

In [13]:
# wr.config.s3_endpoint_url = "http://192.168.1.7:8333"

# # wr.s3.read_parquet(
# #     path=f"s3://partlist_label/",
# #     chunked=100
# # )
# for i in wr.s3.read_parquet(
#     path=f"s3://partlist_label/",
#     chunked=100
# ):
#     print(i.columns)
#     break

In [14]:
wr.config.s3_endpoint_url = "http://192.168.1.7:8333"

labelDf = wr.s3.read_parquet(
    path=f"s3://partlist_label/",
    columns=["CaseID", "lvl_1_desc", "lvl_2_desc", "lvl_3_desc"]
)

In [15]:
caseDf = wr.s3.read_parquet(
    path=f"s3://scope_case/",
    # chunked=100
)
# for i in caseDf:
#     print(i.columns)
#     break

In [16]:
labelDf = labelDf.merge(caseDf, on="CaseID")

In [17]:
allLvl3Labels = labelDf["lvl_3_desc"].unique().tolist()
allLvl2Labels = labelDf["lvl_2_desc"].unique().tolist()


In [18]:
rawMapping = {
    f"vision_{x}" : 0 for x in allLvl3Labels
}
for x in allLvl2Labels:
    rawMapping[x] = 0

In [19]:
def worker(caseIdList, labelDf):
    allPayload = []
    for caseId in tqdm(caseIdList, desc="case"):
        targets = labelDf[labelDf["CaseID"] == caseId]
        
        caseData = copy.deepcopy(rawMapping)
        caseData["CaseID"] = targets.iloc[0]["CaseID"]
        caseData["Circumstances_of_Accident"] = targets.iloc[0]["Circumstances_of_Accident"]
        lvl2DmgParts = targets["lvl_2_desc"].tolist()
        lvl3DmgParts = targets["lvl_3_desc"].tolist()

        for part in lvl3DmgParts:
            caseData[f"vision_{part}"] = 1
        for part in lvl2DmgParts:
            caseData[part] = 1

        allPayload.append(caseData)
    return pd.json_normalize(allPayload)

In [20]:
allPayload = []
allDf = []
allCaseId = labelDf["CaseID"].unique().tolist()
tasks = []
batchSize = 10000
for i in range(0, len(allCaseId), batchSize):
    tasks.append(allCaseId[i:i + batchSize])
result = Parallel(n_jobs=3)(delayed(worker)(
        task, labelDf) for task in tqdm(tasks))

 32%|███▏      | 20/63 [01:24<03:33,  4.97s/it]

KeyboardInterrupt: 

In [ ]:
multilabelDf = pd.concat(allDf)
multilabelDf

,vision_bonnet,vision_bumper front,vision_grille,vision_headlamp rh,vision_headlamp lh,vision_door front lh,vision_door front rh,vision_engine,vision_bumper rear,vision_misc,...,h,d,c,vision_s,vision_c,s,vision_w,w,vision_q,q
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0,0,0,0,0,0,0,0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999996,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999997,0,0,0,0,0,0,0,0,0,0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
999998,0,0,0,0,0,0,0,0,0,0,...,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
999999,0,0,0,0,0,0,0,0,0,0,...,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
cli = boto3.client("s3", **{
    "endpoint_url": "http://192.168.1.7:8333",
    "aws_access_key_id": "",
    "aws_secret_access_key": "",
    # "Username": "aaa",
})
outputBucketName = "multilabel_df"
# cli.create_bucket(Bucket=outputBucketName)
wr.s3.to_parquet(
                    df=multilabelDf,
                    path=f"s3://{outputBucketName}/",
                    dataset=True,
                    mode="overwrite",
                )

{'paths': ['s3://multilabel_df/709c2e2eb4324ecf866f5ef8ae755fac.snappy.parquet'],
 'partitions_values': {}}